## Getting access to data

In [ ]:
from stravalib import Client
import pickle
import time
import folium
import pandas as pd

client = Client()

MY_STRAVA_CLIENT_ID, MY_STRAVA_CLIENT_SECRET = open('client.secret').read().strip().split(',')
print ('Client ID and secret read from file'.format(MY_STRAVA_CLIENT_ID) )

In [ ]:
with open('access_token.pickle', 'rb') as f:
    access_token = pickle.load(f)
    
print('Latest access token read from file:')
access_token

In [ ]:
if time.time() > access_token['expires_at']:
    print('Token has expired, will refresh')
    refresh_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID, 
                                               client_secret=MY_STRAVA_CLIENT_SECRET, 
                                               refresh_token=access_token['refresh_token'])
    access_token = refresh_response
    with open('../access_token.pickle', 'wb') as f:
        pickle.dump(refresh_response, f)
    print('Refreshed token saved to file')

    client.access_token = refresh_response['access_token']
    client.refresh_token = refresh_response['refresh_token']
    client.token_expires_at = refresh_response['expires_at']
        
else:
    print('Token still valid, expires at {}'.format(time.strftime("%a, %d %b %Y %H:%M:%S %Z", time.localtime(access_token['expires_at']))))

    client.access_token = access_token['access_token']
    client.refresh_token = access_token['refresh_token']
    client.token_expires_at = access_token['expires_at']

In [ ]:
athlete = client.get_athlete()
print("Athlete's name is {} {}, based in {}, {}"
      .format(athlete.firstname, athlete.lastname, athlete.city, athlete.country))

## Creating the heatmap

In [ ]:
activities = client.get_activities(limit=20)
my_cols =['name',
          'start_date_local',
          'type',
          'distance',
          'moving_time',
          'elapsed_time',
          'total_elevation_gain',
          'elev_high',
          'elev_low',
          'average_speed',
          'max_speed',
          'average_heartrate',
          'max_heartrate',
          'start_latitude',
          'start_longitude']
data = []
for activity in activities:
    my_dict = activity.to_dict()
    data.append([activity.id]+[my_dict.get(x) for x in my_cols])
    
# Add id to the beginning of the columns, used when selecting a specific activity
my_cols.insert(0,'id')

df = pd.DataFrame(data, columns=my_cols)

In [ ]:
df.head()

In [ ]:
mymap = folium.Map(zoom_start=12, tiles='CartoDB dark_matter')
color_per_type = {'Ride': 'blue', 'Hike': 'red', 'Run': 'red'}

for id, type in zip(df['id'], df['type']):
    try:
        types = ['time', 'distance', 'latlng', 'altitude', 'velocity_smooth', 'moving', 'grade_smooth']
        activity_data=client.get_activity_streams(id, types=types)
        if type in color_per_type.keys():
            color = color_per_type[type]
        else:
            color = 'yellow'
        folium.PolyLine(activity_data['latlng'].data, color=color, opacity=0.5, weight=2).add_to(mymap)
    except:
        print(id, type)


In [ ]:
mymap